In [1]:
!pip install evaluate==0.4.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [3]:
import numpy as np

# This is for consistency, you can ignore this
np.object = object

# Get to know the data

In [4]:
import pandas as pd
from datasets import Dataset

dataset_path_train = "/kaggle/input/amazon-beverage-reviews-with-features-rating/train.csv"
dataset_train = pd.read_csv(dataset_path_train)
review1_data = dataset_train[['reviewText_1','appearanceRate_1', 'aromaRate_1', 'palateRate_1', 'tasteRate_1']]
review2_data = dataset_train[['reviewText_2','appearanceRate_2', 'aromaRate_2', 'palateRate_2', 'tasteRate_2']]

review2_data = review2_data.rename(columns={'reviewText_2': 'reviewText', 'appearanceRate_2': 'appearanceRate',
                                            'aromaRate_2': 'aromaRate', 'palateRate_2': 'palateRate', 'tasteRate_2': 'tasteRate'})

review1_data = review1_data.rename(columns={'reviewText_1': 'reviewText', 'appearanceRate_1': 'appearanceRate',
                                            'aromaRate_1': 'aromaRate', 'palateRate_1': 'palateRate', 'tasteRate_1': 'tasteRate'})

df = pd.concat([review1_data,review2_data], axis=0)

df["appearanceRate"] = df["appearanceRate"] * 2 - 2
df["appearanceRate"] = df["appearanceRate"].astype(int)

df["aromaRate"] = df["aromaRate"] * 2 - 2
df["aromaRate"] = df["aromaRate"].astype(int)

df["palateRate"] = df["palateRate"] * 2 - 2
df["palateRate"] = df["palateRate"].astype(int)

df["tasteRate"] = df["tasteRate"] * 2 - 2
df["tasteRate"] = df["tasteRate"].astype(int)

ds_train = Dataset.from_pandas(df,split="train")
ds_train = ds_train.remove_columns("__index_level_0__")

ds_train

Dataset({
    features: ['reviewText', 'appearanceRate', 'aromaRate', 'palateRate', 'tasteRate'],
    num_rows: 40000
})

In [5]:
dataset_path_val = "/kaggle/input/amazon-beverage-reviews-with-features-rating/val.csv"
dataset_val = pd.read_csv(dataset_path_val)

review1_data = dataset_val[['reviewText_1','appearanceRate_1', 'aromaRate_1', 'palateRate_1', 'tasteRate_1']]
review2_data = dataset_val[['reviewText_2','appearanceRate_2', 'aromaRate_2', 'palateRate_2', 'tasteRate_2']]

review2_data = review2_data.rename(columns={'reviewText_2': 'reviewText', 'appearanceRate_2': 'appearanceRate',
                                            'aromaRate_2': 'aromaRate', 'palateRate_2': 'palateRate', 'tasteRate_2': 'tasteRate'})

review1_data = review1_data.rename(columns={'reviewText_1': 'reviewText', 'appearanceRate_1': 'appearanceRate',
                                            'aromaRate_1': 'aromaRate', 'palateRate_1': 'palateRate', 'tasteRate_1': 'tasteRate'})

df = pd.concat([review1_data,review2_data], axis=0)

df["appearanceRate"] = df["appearanceRate"] * 2 - 2
df["appearanceRate"] = df["appearanceRate"].astype(int)

df["aromaRate"] = df["aromaRate"] * 2 - 2
df["aromaRate"] = df["aromaRate"].astype(int)

df["palateRate"] = df["palateRate"] * 2 - 2
df["palateRate"] = df["palateRate"].astype(int)

df["tasteRate"] = df["tasteRate"] * 2 - 2
df["tasteRate"] = df["tasteRate"].astype(int)


ds_val = Dataset.from_pandas(df,split="validation")
ds_val = ds_val.remove_columns("__index_level_0__")

ds_val 


Dataset({
    features: ['reviewText', 'appearanceRate', 'aromaRate', 'palateRate', 'tasteRate'],
    num_rows: 15000
})

In [6]:
from datasets import DatasetDict

raw_dataset = DatasetDict({"train": ds_train, "validation": ds_val})
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['reviewText', 'appearanceRate', 'aromaRate', 'palateRate', 'tasteRate'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['reviewText', 'appearanceRate', 'aromaRate', 'palateRate', 'tasteRate'],
        num_rows: 15000
    })
})

In [7]:
raw_dataset['train'][25]

{'reviewText': "12oz. bottle poured into my trusty mug. A- pretty bright gold with orange hints. nice white head, that leaves a little lacing down the glass. S- a light, sweet caramel malt with some floral based hops T- Great googly moogly. Taste-wise, possibly the finest Pale I've ever had the pleasure of tasting. The malt bill is more complex on the taste, taking on the likeness of an ESB. Caramel, biscuit and tea mostly. The floral hops are a bit citric, but are absolutely spot on in their assertiveness, balancing the brew. M- Great on the palette. Medium bodied and slick, with a light carbonation, and it has a pleasently dry finish. O- I really, really wish I could get this in Eastern PA, as this would be my go-to Pale. Fantastic.",
 'appearanceRate': 6,
 'aromaRate': 6,
 'palateRate': 7,
 'tasteRate': 8}

In [8]:
raw_dataset_df = raw_dataset['train'].to_pandas()
raw_dataset_df.head()

,reviewText,appearanceRate,aromaRate,palateRate,tasteRate
0,had to have this beer for several times tonigh...,0,1,1,0
1,Poured from 12 oz. bottle into pint glass. Lig...,5,5,5,5
2,I tried this beer along with their oak aged ve...,6,6,6,7
3,Poured into a pilsner like small glass. Dark b...,4,4,4,3
4,Poured from a 12oz. bottle into a my Chimay Ch...,3,4,6,3


In [9]:
features = raw_dataset['train'].features
features

{'reviewText': Value(dtype='string', id=None),
 'appearanceRate': Value(dtype='int64', id=None),
 'aromaRate': Value(dtype='int64', id=None),
 'palateRate': Value(dtype='int64', id=None),
 'tasteRate': Value(dtype='int64', id=None)}

# Tokenize the data

In [10]:
from transformers import AutoTokenizer
check_point = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(check_point)
tokenizer(raw_dataset['train']['reviewText'][0])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 2018, 2000, 2031, 2023, 5404, 2005, 2195, 2335, 3892, 1010, 2245, 1045, 2323, 3319, 1012, 8542, 2013, 11112, 2046, 1037, 9231, 2102, 1010, 7708, 9231, 2102, 2130, 1012, 3504, 1010, 14747, 1010, 5510, 1010, 2066, 2673, 1045, 18959, 2013, 1037, 5404, 1012, 1996, 2069, 2204, 2518, 1045, 2064, 2360, 2003, 2008, 1996, 2132, 20125, 2003, 3243, 3835, 1012, 2021, 1996, 2132, 2987, 2102, 3104, 1996, 5510, 2030, 23958, 1012, 2025, 1996, 5409, 5404, 1045, 1005, 2310, 2018, 1010, 2021, 2028, 1997, 1044, 2618, 5409, 1045, 1005, 2310, 2018, 9906, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [11]:
def tokenize_func(examples):
  return tokenizer(examples["reviewText"], truncation=True, max_length=512, padding= True)

In [12]:
tokenized_dataset = raw_dataset.map(tokenize_func, batched=True)
# Only consider taste aspect
tokenized_dataset = tokenized_dataset.remove_columns(["appearanceRate", "palateRate", "aromaRate", "reviewText"])
tokenized_dataset = tokenized_dataset.rename_column("tasteRate", "c")
tokenized_dataset = tokenized_dataset.with_format("torch")

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [13]:
df = tokenized_dataset['train'].to_pandas()
df.head(200)

,labels,input_ids,token_type_ids,attention_mask
0,0,"[101, 2018, 2000, 2031, 2023, 5404, 2005, 2195...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,5,"[101, 8542, 2013, 2260, 11472, 1012, 5835, 204...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,7,"[101, 1045, 2699, 2023, 5404, 2247, 2007, 2037...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,3,"[101, 8542, 2046, 1037, 14255, 4877, 3678, 206...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,3,"[101, 8542, 2013, 1037, 2260, 18153, 1012, 583...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
195,5,"[101, 2092, 2144, 1045, 1005, 1049, 2013, 5773...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
196,2,"[101, 17012, 1012, 1012, 1012, 2023, 2003, 103...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
197,6,"[101, 4207, 1037, 5835, 2007, 2627, 12173, 531...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
198,5,"[101, 2023, 5404, 3504, 3492, 2172, 2066, 1037...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


# Train the model

In [14]:
from huggingface_hub import notebook_login

notebook_login()

In [15]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import evaluate
import numpy as np

id2label = {0: "LABEL_1", 1: "LABEL_1.5", 2: "LABEL_2", 3: "LABEL_2.5", 4: "LABEL_3", 5: "LABEL_3.5", 6: "LABEL_4", 7: "LABEL_4.5", 8: "LABEL_5",}
label2id = {'LABEL_1': 0, 'LABEL_1.5': 1, 'LABEL_2': 2, 'LABEL_2.5': 3, 'LABEL_3': 4, 'LABEL_3.5': 5, 'LABEL_4': 6, 'LABEL_4.5': 7, 'LABEL_5': 8}
# make Classlabel
training_args = TrainingArguments("Taste-classifier-with-ratings", evaluation_strategy="epoch", push_to_hub=True, save_strategy="no")
model = AutoModelForSequenceClassification.from_pretrained(check_point, num_labels=9, id2label=id2label, label2id=label2id)




2024-04-13 15:08:55.005243: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 15:08:55.005359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 15:08:55.141484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def compute_metrics(eval_pred):
    metric1 = evaluate.load("accuracy")
    metric2 = evaluate.load("recall")
    metric3 = evaluate.load("precision")
    metric4 = evaluate.load("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
    recall = metric2.compute(predictions=predictions, references=labels, average="micro")["recall"]
    precision = metric3.compute(predictions=predictions, references=labels, average="micro")["precision"]
    f1 = metric4.compute(predictions=predictions, references=labels, average="micro")["f1"]
    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [17]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
# Fine-tuning the model
import wandb
wandb.init(mode="disabled")

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,1.334200,1.386669,0.410333,0.410333,0.410333,0.410333
2,1.152500,1.334645,0.449867,0.449867,0.449867,0.449867
3,0.899700,1.450336,0.449867,0.449867,0.449867,0.449867


TrainOutput(global_step=15000, training_loss=1.1842432556152345, metrics={'train_runtime': 7276.75, 'train_samples_per_second': 16.491, 'train_steps_per_second': 2.061, 'total_flos': 3.157531103232e+16, 'train_loss': 1.1842432556152345, 'epoch': 3.0})

In [19]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1713020949.427ae9b2499d.34.0:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lengocquangLAB/Taste-classifier-with-ratings/commit/8d9830e95f550ae4847cb58f8ec61e3228cd56ce', commit_message='End of training', commit_description='', oid='8d9830e95f550ae4847cb58f8ec61e3228cd56ce', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
Epoch 1/5
40/40 [==============================] - 100s 1s/step - loss: 1.9680 - accuracy: 0.4781 - precision: 0.7009 - recall: 0.2453 - f1-macro: 0.0590 - f1-micro: 0.4781 - val_loss: 1.4683 - val_accuracy: 0.6313 - val_precision: 0.6329 - val_recall: 0.6250 - val_f1-macro: 0.0553 - val_f1-micro: 0.6313
Epoch 2/5
40/40 [==============================] - 35s 866ms/step - loss: 1.1048 - accuracy: 0.6812 - precision: 0.7565 - recall: 0.6359 - f1-macro: 0.0685 - f1-micro: 0.6812 - val_loss: 1.2161 - val_accuracy: 0.6812 - val_precision: 0.7000 - val_recall: 0.6125 - val_f1-macro: 0.0646 - val_f1-micro: 0.6812
Epoch 3/5
40/40 [==============================] - 38s 947ms/step - loss: 0.9102 - accuracy: 0.7188 - precision: 0.8494 - recall: 0.6344 - f1-macro: 0.1081 - f1-micro: 0.7188 - val_loss: 1.2229 - val_accuracy: 0.6687 - val_precision: 0.7500 - val_recall: 0.5437 - val_f1-macro: 0.1252 - val_f1-micro: 0.6687
Epoch 4/5
40/40 [==============================] - 36s 903ms/step - loss: 0.7845 - accuracy: 0.7578 - precision: 0.8870 - recall: 0.6500 - f1-macro: 0.1519 - f1-micro: 0.7578 - val_loss: 1.2046 - val_accuracy: 0.6750 - val_precision: 0.7440 - val_recall: 0.5813 - val_f1-macro: 0.1245 - val_f1-micro: 0.6750
Epoch 5/5
40/40 [==============================] - 37s 918ms/step - loss: 0.6707 - accuracy: 0.7906 - precision: 0.9146 - recall: 0.6859 - f1-macro: 0.1963 - f1-micro: 0.7906 - val_loss: 1.2870 - val_accuracy: 0.6750 - val_precision: 0.7460 - val_recall: 0.5875 - val_f1-macro: 0.1590 - val_f1-micro: 0.6750

In [20]:
from transformers import pipeline

appearance_text_classifier = pipeline("text-classification", model="lengocquangLAB/Taste-classifier-with-ratings")
results = appearance_text_classifier("I love this")
results

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'LABEL_5', 'score': 0.9286513328552246}]